<a href="https://colab.research.google.com/github/PaleSnakee/data-summarization-and-optimization-with-IBM-Granite/blob/main/Copy_of_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community
!pip install replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata
# Set the API token
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token
# Model setup
model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
model=model,
replicate_api_token=api_token,
)

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdulmalik1518/cars-datasets-2025")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cars-datasets-2025


In [ ]:
import os

print(os.listdir(path))

['Cars Datasets 2025.csv']


In [ ]:
import pandas as pd

df = pd.read_csv(f"{path}/Cars Datasets 2025.csv", encoding='latin-1')

print(df.head())  # show 5 data head

  Company Names     Cars Names      Engines CC/Battery Capacity HorsePower  \
0       FERRARI  SF90 STRADALE           V8             3990 cc     963 hp   
1   ROLLS ROYCE        PHANTOM          V12             6749 cc     563 hp   
2          Ford            KA+  1.2L Petrol            1,200 cc   70-85 hp   
3      MERCEDES        GT 63 S           V8            3,982 cc     630 hp   
4          AUDI     AUDI R8 Gt          V10            5,204 cc     602 hp   

  Total Speed Performance(0 - 100 )KM/H      Cars Prices       Fuel Types  \
0    340 km/h                   2.5 sec      $1,100,000   plug in hyrbrid   
1    250 km/h                   5.3 sec        $460,000            Petrol   
2    165 km/h                  10.5 sec  $12,000-$15,000           Petrol   
3    250 km/h                   3.2 sec        $161,000            Petrol   
4    320 km/h                   3.6 sec        $253,290            Petrol   

  Seats        Torque  
0     2        800 Nm  
1     5        900 N

In [ ]:
# Refine the prompt to include reviews
reviews_text = df.to_csv(index=False)

# Set model parameters for prompting with default values
parameters = {
 "top_k": 10,
 "top_p": 1,
 "max_tokens": 1200,
 "min_tokens": 0,
 "random_seed": None,
 "repetition_penalty": 1.5,
 "stopping_criteria": None,
 "stopping_sequence": "\n"
}

# Add initial prompt
refined_prompt = f""":
Classify each car in the dataset into categories based on its specifications.
Classification rules:
Performance Class (based on Top Speed and Acceleration):
Hypercar: Top Speed ≥ 320 km/h and Acceleration ≤ 3.5 sec
Supercar: 250–319 km/h and Acceleration ≤ 4.5 sec
Sports: 200–249 km/h
Economy: < 200 km/h

Engine Size Category:
Small: < 2.0 L
Medium: 2.0–4.0 L
Large: > 4.0 L

Price Segment:
Entry-level: < $50,000
Mid-range: $50,000–$200,000
Luxury: > $200,000

Fuel Type Group: Petrol, Diesel, Hybrid, Electric.

Output requirement:
provide a table with column: Car Name, Company, Performance Class, Engine Size Category, Price Segment, Fuel Type Group.
after the table, give a short summary of how many cars fall into each category for every classification dimension.
{reviews_text}
"""
# Invoke the model
response = output.invoke(refined_prompt, parameters=parameters)
# Print the response
print("Granite Model Refined Response:\n")
print(response)

Granite Model Refined Response:

| Car Name                     | Company          | Performance Class                   | Engine Size Category | Price Segment          | Fuel Type Group               |
|-------------------------------|------------------|----------------------------------|---------------------|----------------------|---------------------------------|
| SF90 Stradale                | Ferrari         | Hypercar                          | Large               | Luxury               | Petrol                          |
| Phantom                     | Rolls Royce     | Luxury                            | Large               | Luxury               | Petrol                          |
| KA+                         | Ford            | Economy                           | Small               | Entry-level          | Petrol                          |
| GT 63 S                      | Mercedes        | Supercar                          | Medium              | Luxury               | Pe